In [1]:
using StatisticalRethinking, Distributed, JLD
using Mamba

# Data
line = Dict{Symbol, Any}()

howell1 = CSV.read(rel_path("..", "data", "Howell1.csv"), delim=';')
df = convert(DataFrame, howell1);

Use only adults

In [2]:
df2 = filter(row -> row[:age] >= 18, df);
line[:x] = convert(Array{Float64,1}, df2[:weight]);
line[:y] = convert(Array{Float64,1}, df2[:height]);
line[:xmat] = convert(Array{Float64,2}, [ones(length(line[:x])) line[:x]])

# Model Specification
model = Model(
  y = Stochastic(1,
    (xmat, beta, s2) -> MvNormal(xmat * beta, sqrt(s2)),
    false
  ),
  beta = Stochastic(1, () -> MvNormal([178, 0], [sqrt(10000), sqrt(100)])),
  s2 = Stochastic(() -> Uniform(0, 50))
)

# Initial Values
inits = [
  Dict{Symbol, Any}(
    :y => line[:y],
    :beta => [rand(Normal(178, 100)), rand(Normal(0, 10))],
    :s2 => rand(Uniform(0, 50))
  )
  for i in 1:3
]

# Tuning Parameters
scale1 = [0.5, 0.25]
summary1 = identity
eps1 = 0.5

scale2 = 0.5
summary2 = x -> [mean(x); sqrt(var(x))]
eps2 = 0.1

# Define sampling scheme

scheme = [
  Mamba.NUTS([:beta]),
  Mamba.Slice([:s2], 10)
]

setsamplers!(model, scheme)

# MCMC Simulation

sim = mcmc(model, line, inits, 10000, burnin=1000, chains=3)

MCMC Simulation of 10000 Iterations x 3 Chains...

Chain 1:   0% [0:23:08 of 0:23:09 remaining]
Chain 1:  10% [0:01:17 of 0:01:25 remaining]
Chain 1:  20% [0:01:05 of 0:01:21 remaining]
Chain 1:  30% [0:00:58 of 0:01:23 remaining]
Chain 1:  40% [0:00:49 of 0:01:21 remaining]
Chain 1:  50% [0:00:41 of 0:01:22 remaining]
Chain 1:  60% [0:00:33 of 0:01:22 remaining]
Chain 1:  70% [0:00:25 of 0:01:23 remaining]
Chain 1:  80% [0:00:17 of 0:01:24 remaining]
Chain 1:  90% [0:00:09 of 0:01:26 remaining]
Chain 1: 100% [0:00:00 of 0:01:25 remaining]

Chain 2:   0% [0:00:03 of 0:00:03 remaining]
Chain 2:  10% [0:00:54 of 0:01:00 remaining]
Chain 2:  20% [0:00:52 of 0:01:05 remaining]
Chain 2:  30% [0:00:47 of 0:01:07 remaining]
Chain 2:  40% [0:00:41 of 0:01:09 remaining]
Chain 2:  50% [0:00:35 of 0:01:10 remaining]
Chain 2:  60% [0:00:28 of 0:01:11 remaining]
Chain 2:  70% [0:00:22 of 0:01:15 remaining]
Chain 2:  80% [0:00:15 of 0:01:17 remaining]
Chain 2:  90% [0:00:08 of 0:01:19 remaining]
Cha

Object of type "Mamba.ModelChains"

Iterations = 1001:10000
Thinning interval = 1
Chains = 1,2,3
Samples per chain = 9000

[24.4225 112.563 0.939842; 26.2636 114.695 0.881133; … ; 25.1682 111.967 0.950843; 25.9559 111.967 0.950843]

[27.196 119.023 0.789815; 27.3217 119.238 0.788734; … ; 28.2215 112.126 0.945856; 30.6483 112.317 0.94303]

[28.0833 113.451 0.914718; 21.7313 113.45 0.907421; … ; 26.0943 113.691 0.913527; 28.7337 113.692 0.902021]

Show draws summary

In [3]:
describe(sim)

Iterations = 1001:10000
Thinning interval = 1
Chains = 1,2,3
Samples per chain = 9000

Empirical Posterior Estimates:
            Mean        SD       Naive SE        MCSE        ESS   
     s2  26.1708747 2.0048111 0.01220089156 0.02083985639 9000.0000
beta[1] 113.8712713 1.9376044 0.01179188475 0.03781875600 2624.9180
beta[2]   0.9052598 0.0425850 0.00025916406 0.00082326172 2675.7002

Quantiles:
            2.5%        25.0%       50.0%       75.0%        97.5%   
     s2  22.53556428  24.7805631  26.0650831  27.4602841  30.36419663
beta[1] 109.94981054 112.5565543 113.8695398 115.1949629 117.59127045
beta[2]   0.82376649   0.8760951   0.9055728   0.9340105   0.99114626



End of `clip_38.0m.jl`

*This notebook was generated using [Literate.jl](https://github.com/fredrikekre/Literate.jl).*